# Flower Classfication with CNN

## Overview

We design and train a deep convolutional network from scratch to predict the class label of a flower image.

##  Versions of used packages
`python 3.8.11`, `torch==1.8.2` and `torchvision==0.9.2`

In [ ]:
import sys
import torch
import torchvision
print('python', sys.version.split('\n')[0])
print('torch', torch.__version__)
print('torchvision', torchvision.__version__)

python 3.7.12 (default, Sep 10 2021, 00:21:48) 
torch 1.10.0+cu111
torchvision 0.11.1+cu111


# Dataset

We use [Flowers Recognition](https://www.kaggle.com/alxmamaev/flowers-recognition) dataset, which collected by Alexander Mamaev.

Flowers Recognition dataset contains 4317 flower images.  

The pictures are divided into five classes: 
+ daisy
+ tulip
+ rose
+ sunflower
+ dandelion

For each class there are about 800 photos. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -qq ./drive/MyDrive/Flowers Recognition.zip

In [ ]:
data_folder = 'Flowers Recognition'

## Loading the dataset


In [ ]:
import csv
import os
import numpy as np
from PIL import Image
import torch

class FlowerData(torch.utils.data.Dataset):
    def __init__(self, csv_file, mode='train', transform=None):
        self.mode = mode # 'train', 'val' or 'test'
        self.data_list = []
        self.labels = []
        self.transform = transform
        
        with open(f'{data_folder}/{csv_file}', newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                self.data_list.append(f"{data_folder}/{row['file_path']}")
                if mode != 'test':
                    self.labels.append(row['label'])

    def __getitem__(self, index):
        data = Image.open(self.data_list[index])
        if self.transform is not None:
            data = self.transform(data)
        if self.mode == 'test':
            return data
        label = torch.tensor(int(self.labels[index]))

        return data, label

    def __len__(self):
        return len(self.data_list)

### Data augmentation 

In [ ]:
from torchvision import transforms

# For train
transforms_train = transforms.Compose(
    [transforms.Resize((285, 285)),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomVerticalFlip(p=0.5),
     transforms.RandomRotation((-180, 180)),
     transforms.RandomCrop(256, padding=2),
     transforms.ToTensor(),
     transforms.Normalize((0.48, 0.45, 0.46),(0.25, 0.25, 0.25))]
)


# For val, test
transforms_test = transforms.Compose(
    [transforms.Resize((285, 285)),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize((0.48, 0.45, 0.46),(0.25, 0.25, 0.25))]
)

### Instantiate dataset

Let's instantiate three `FlowerData` class.
+ dataset_train: for training.
+ dataset_val: for validation.
+ dataset_test: for tesing.

In [ ]:
dataset_train = FlowerData('train.csv', mode='train', transform=transforms_train)
dataset_val = FlowerData('val.csv', mode='val', transform=transforms_test)
dataset_test = FlowerData('test.csv', mode='test', transform=transforms_test)

In [ ]:
print("The first image's shape in dataset_train :", dataset_train.__getitem__(0)[0].size())
print("There are", dataset_train.__len__(), "images in dataset_train.")

The first image's shape in dataset_train : torch.Size([3, 256, 256])
There are 1295 images in dataset_train.


### DataLoader

In [ ]:
from torch.utils.data import DataLoader

batch_size = 32
num_workers = 2
train_loader = DataLoader(dataset_train, batch_size=batch_size, num_workers=num_workers, shuffle=True)
val_loader = DataLoader(dataset_val, batch_size=batch_size, num_workers=num_workers, shuffle=False)
test_loader = DataLoader(dataset_test, batch_size=batch_size, num_workers=num_workers, shuffle=False)

# Build Model

### Convolutional Neural Network

In [ ]:
import torch.nn as nn 
import torch.nn.functional as F

class Your_CNN_Model(nn.Module): 
    def __init__(self): 
        super().__init__()


        # CNN model
        self.conv1 = nn.Conv2d(3, 32, 3, padding=2)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)

        self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv4 = nn.Conv2d(32, 64, 3, padding=1)  


        self.conv5 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv6 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv7 = nn.Conv2d(128, 128, 3, padding=1)


        self.conv8 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv9 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv10 = nn.Conv2d(256, 512, 3, padding=1)

        self.fc1 = nn.Linear(512 * 16 * 16, 600)
        self.fc2 = nn.Linear(600, 200)
        self.fc3 = nn.Linear(200, 5)

        self.dropout1 = nn.Dropout(0.4)


    def forward(self, x): 
        if not isinstance(x, torch.Tensor):
            x = torch.Tensor(x)
 
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(F.relu(self.conv2(out)), 2)

        out = F.relu(self.conv3(out))
        out = F.max_pool2d(F.relu(self.conv4(out)), 2)

        out = F.relu(self.conv5(out))
        out = F.relu(self.conv6(out))
        out = F.max_pool2d(F.relu(self.conv7(out)), 2)

        out = F.relu(self.conv8(out))
        out = F.relu(self.conv9(out))
        out = F.max_pool2d(F.relu(self.conv10(out)), 2)

        out = out.view(out.size(0), -1)
        out = torch.flatten(out, 1)
        out = F.relu(self.fc1(out))
        out = self.dropout1(out)
        out = F.relu(self.fc2(out))
        out = self.dropout1(out)
        out = self.fc3(out)

        return out

In [ ]:
model = Your_CNN_Model()
model = model.cuda()
print(model)

Your_CNN_Model(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv9): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv10): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=131072, out_features=600, bias=True)
  (fc2): Linear(in_features=600, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=5, bias=True)
  (dropout1): Drop

In [ ]:
device = torch.device('cuda')
# device = torch.device('cpu')

In [ ]:
model = Your_CNN_Model()
model = model.to(device)
# print(model)

### Define loss and optimizer

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Define loss and optmizer 
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr = 0.0005)

criterion = criterion.to(device)

### Train Function

In [ ]:
def train(input_data, model, criterion, optimizer):
    '''
    Argement:
    input_data -- iterable data, typr torch.utils.data.Dataloader is prefer
    model -- nn.Module, model contain forward to predict output
    criterion -- loss function, used to evaluate goodness of model
    optimizer -- optmizer function, method for weight updating
    '''
    model.train()
    loss_list = []
    total_count = 0
    acc_count = 0
    for images, labels in input_data:
        images = images.to(device)
        labels = labels.to(device)
        

        # Forward, backward and optimize                                 
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
      

        # Get the counts of correctly classified images
        _, predicted = torch.max(outputs.data, 1)
        total_count += labels.size(0)
        acc_count += (predicted == labels).sum().item()
        loss_list.append(loss.item())
  

    # Compute this epoch accuracy and loss
    acc = acc_count / total_count
    loss = sum(loss_list) / len(loss_list)
    return acc, loss

#### Validate function

In [ ]:
def val(input_data, model, criterion):
    model.eval()
    
    loss_list = []
    total_count = 0
    acc_count = 0
    with torch.no_grad():
        for images, labels in input_data:
            images = images.to(device)
            labels = labels.to(device)

            # Get the predicted result and loss
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total_count += labels.size(0)
            acc_count += (predicted == labels).sum().item()
            loss_list.append(loss.item())

    acc = acc_count / total_count
    loss = sum(loss_list) / len(loss_list)
    return acc, loss

## Training

In [ ]:
# hyper parameters
max_epochs = 200
log_interval = 2 


train_acc_list = []
train_loss_list = []
val_acc_list = []
val_loss_list = []      
      
for epoch in range(1, max_epochs + 1):
    train_acc, train_loss = train(train_loader, model, criterion, optimizer)
    val_acc, val_loss = val(val_loader, model, criterion)
    train_acc_list.append(train_acc)
    train_loss_list.append(train_loss)
    val_acc_list.append(val_acc)   
    val_loss_list.append(val_loss)

    # print acc and loss in per log_interval time          
    if epoch % log_interval == 0:    
        print('=' * 20, 'Epoch', epoch, '=' * 20)      
        print('Train Acc: {:.6f} Train Loss: {:.6f}'.format(train_acc, train_loss)) 
        print('  Val Acc: {:.6f}   Val Loss: {:.6f}'.format(val_acc, val_loss)) 


==================== Epoch 2 ====================
Train Acc: 0.230116 Train Loss: 1.603611
  Val Acc: 0.244186   Val Loss: 1.596244
==================== Epoch 4 ====================
Train Acc: 0.299614 Train Loss: 1.529226
  Val Acc: 0.318605   Val Loss: 1.496073
==================== Epoch 6 ====================
Train Acc: 0.322008 Train Loss: 1.465235
  Val Acc: 0.390698   Val Loss: 1.382733
==================== Epoch 8 ====================
Train Acc: 0.423166 Train Loss: 1.307843
  Val Acc: 0.474419   Val Loss: 1.258784
==================== Epoch 10 ====================
Train Acc: 0.481853 Train Loss: 1.159371
  Val Acc: 0.490698   Val Loss: 1.131325
==================== Epoch 12 ====================
Train Acc: 0.522780 Train Loss: 1.100544
  Val Acc: 0.506977   Val Loss: 1.123086
==================== Epoch 14 ====================
Train Acc: 0.547490 Train Loss: 1.070274
  Val Acc: 0.588372   Val Loss: 1.040220
==================== Epoch 16 ====================
Train Acc: 0.579151 Tr

In [ ]:
# save well-trained state_dict of model          
torch.save(model.state_dict(), 'NAME_OF_THIS_EXPERIMENT.pt') 

#### Visualize accuracy and loss

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.plot(range(len(train_loss_list)), train_loss_list)
plt.plot(range(len(val_loss_list)), val_loss_list, c='r')
plt.legend(['train', 'val'])
plt.title('Loss')
plt.show()
plt.figure(figsize=(12, 4))
plt.plot(range(len(train_acc_list)), train_acc_list)
plt.plot(range(len(val_acc_list)), val_acc_list, c='r')
plt.legend(['train', 'val'])
plt.title('Acc')
plt.show()

### Prediction

In [ ]:
## load previous best model
# ckpt = torch.load('NAME_OF_THIS_EXPERIMENT.pt')
# model.load_state_dict(ckpt) 

In [ ]:
def predict(input_data, model):
    model.eval()
    output_list = []
    with torch.no_grad():
        for images in input_data:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            output_list.extend(predicted.to('cpu').numpy().tolist())
    return output_list

In [ ]:
idx = 0
output_csv = predict(test_loader, model)
with open('result.csv', 'w', newline='') as csvFile:
    writer = csv.DictWriter(csvFile, fieldnames=['file_path', 'label'])
    writer.writeheader()
    for result in output_csv:
        file_path = dataset_test.data_list[idx].replace(data_folder + '/', '')
        writer.writerow({'file_path':file_path, 'label':result})
        idx += 1